In [2]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [3]:
SEASONS = list(range(2016, 2023))

In [4]:
DATA_DIR = "data"
STANDING_DIR = os.path.join(DATA_DIR, "standing")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [5]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries):
        time.sleep(sleep * 1)

        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print (await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [ ]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://www.basketball-reference.com{l}" for l in href]

    for url in standings_pages:
        save_path = os.path.join(STANDING_DIR, url.split("/")[-1])
        if os.path.exists(save_path)
            continue

    html = await get_html(url, "#all_schedule")
    with open (save_path, "w+") as f:
        f.write(html)

In [ ]:
for season in SEASONS:
    await scrape_season(season)